# ME449 - Homework 3 - Zhengyang Kris Weng Submission

In [4]:
import numpy as np
import modern_robotics as mr
from tqdm import tqdm
import ur5_parameters
ur5 = ur5_parameters.UR5()

## Part0: Puppet function

## Part 1: Simulating a falling robot. 
In the first part, the robot will fall in gravity without
damping or the external spring (joint damping and spring stiffness are set to zero). Since there is
no damping or friction, the total energy of the robot (kinetic plus potential) should remain constant
during motion. Gravity is g = 9.81 m/s2 in the −ˆzs-direction, i.e., gravity acts downward.
Simulate the robot falling from rest at the home configuration for five seconds. The output data
should be saved as a .csv file, where each of the N rows has six numbers separated by commas.
This .csv file is suitable for animation with the CoppeliaSim UR5 csv animation scene. Adjust
the animation scene playback speed (“Time Multiplier”) so it takes roughly five seconds of wall
clock time to play your csv file. You can evaluate if your simulation is preserving total energy by
visually checking if the robot appears to swing to the same height (same potential energy) each
swing. Choose values of dt (a) where the energy appears nearly constant (without choosing dt
unnecessarily small) and (b) where the energy does not appear constant (because your timestep
is too coarse). Capture a video for each case and note the dt chosen for each case. Explain how
you would calculate the total energy of the robot at each timestep if you wanted to plot the total
energy to confirm that your simulation approximately preserves it.

In [5]:
# Puppet function:
def puppet_q1(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q1: free falling in gravity

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, np.zeros(n), g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [6]:
q1_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q1_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q1_thetamat, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.001, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat.csv', q1_thetamat, delimiter=',')

100%|██████████| 5000/5000 [00:32<00:00, 156.21it/s]


In [7]:
q1_thetamat_coarse, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat_coarse.csv', q1_thetamat_coarse, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.37it/s]


Loooking at video part1a and part1b.mp4, we can see that energy is not conserved due to error in numerical integration when timestep is too coarse. A good measure of total system energy would be the Hamiltonian of the system, as a sum of potential energy and kinetic energy of each link. By plotting the trend of Hamiltonian of each configuration in the trajectory, this will visualize the preservation of energy in the system. [TODO: IMPLEMENT COMPUTE H WHEN HAVE TIME]

## Part 2: Adding damping. 
Now experiment with different damping coefficients as the robot
falls from the home configuration. Damping causes a torque at each joint equal to the negative of
the joint rate times the damping. Create two videos showing that (a) when you choose damping to
be positive, the robot loses energy as it swings, and (b) when you choose damping to be negative,
the robot gains energy as it swings. Use t = 5 s and dt = 0.01 s, and for the case of positive
damping, the damping coefficient should almost (but not quite) bring the robot to rest by the end
of the video. Do you see any strange behavior in the simulation if you choose the damping constant
to be a large positive value? Can you explain it? How would this behavior change if you chose
shorter simulation timesteps?

In [8]:
def puppet_q2(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q2: Adding damping to robot. Damping causes a tau = - dtheta * damping

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        tau_damping = - damping * dthetalist
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [9]:
q2_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q2_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q2_thetamat, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 1.2, 0, 0)
# Save to csv file
np.savetxt('q2_thetamat.csv', q2_thetamat, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.30it/s]


In [10]:
q2_thetamat_neg, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, -0.02, 0, 0)
# Save to csv file
print(q2_thetamat_neg)
np.savetxt('q2_thetamat_neg.csv', q2_thetamat_neg, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.56it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [-3.07812861e-01 -9.92195181e+00  1.31700123e+01 -6.81968171e+00
  -8.36251555e-01  5.89264993e+01]
 [-3.02987560e-01 -1.00110250e+01  1.32499198e+01 -6.84260777e+00
  -8.76917115e-01  5.96387109e+01]
 [-2.97193787e-01 -1.00998345e+01  1.33216146e+01 -6.85772600e+00
  -9.17977817e-01  6.03581957e+01]]


In [11]:
q2_thetamat_large, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 20, 0, 0)
# Save to csv file
print(q2_thetamat_large)
np.savetxt('q2_thetamat_large.csv', q2_thetamat_large, delimiter=',')

  0%|          | 0/500 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/modern_robotics/core.py:927: RuntimeWarning: overflow encountered in multiply
  + np.dot(ad(Vi[:, i + 1]), Ai[:, i]) * dthetalist[i]
  3%|▎         | 17/500 [00:00<00:02, 165.73it/s]/usr/lib/python3/dist-packages/modern_robotics/core.py:143: RuntimeWarning: invalid value encountered in sin
  return np.eye(3) + np.sin(theta) * omgmat \
/usr/lib/python3/dist-packages/modern_robotics/core.py:144: RuntimeWarning: invalid value encountered in cos
  + (1 - np.cos(theta)) * np.dot(omgmat, omgmat)
/usr/lib/python3/dist-packages/modern_robotics/core.py:366: RuntimeWarning: invalid value encountered in multiply
  np.dot(np.eye(3) * theta \
/usr/lib/python3/dist-packages/modern_robotics/core.py:367: RuntimeWarning: invalid value encountered in cos
  + (1 - np.cos(theta)) * omgmat \
/usr/lib/python3/dist-packages/modern_robotics/core.py:368: RuntimeWarning: invalid value encountered in sin
  + (theta - np.sin(theta)) \
100%|████

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]]


As I increase the magnitude of damping coefficient in the simulation, the function start to run into numerical stability issues and starts to produce nan values in output. This happend because of the numerical instability in the euler integration and the fact that coarse timestep landing next iteration on gradients amplifying such effect; as a result simulation output grew at an extremely fast rate and overflowed. Increasing granularity in timestep helps addressing this issue.

## Part 3: Adding a spring. 
Make gravity and damping zero and design referencePos to return
a constant springPos at (0, 1, 1) in the {s} frame. The spring’s restLength is zero. Experiment
with different stiffness values, and simulate the robot for t = 10 s and dt = 0.01 s starting from
the home configuration. (a) Capture a video for a choice of stiffness that makes the robot oscillate
a couple of times and record the stiffness value. Considering the system’s total energy, does the
motion of the robot make sense? What do you expect to happen to the total energy over time?
Describe the strange behavior you see if you choose the spring constant to be large; if you don’t
see any strange behavior, explain why. (b) Now add a positive damping to the simulation that
makes the arm nearly come to rest by the end of the video. For both videos, record the stiffness
and damping you used.

In [77]:
def puppet_q3(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q3: Adding a static spring.

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        # Calculate damping
        print(f"Iteration {i}")
        tau_damping = - damping * dthetalist
        # Calculate spring force
        spring_force_vec = calculate_spring_wrench(thetalist, Slist, stiffness, restLength, referencePos_q3(i*dt))
        print(spring_force_vec)
        # Forward dynamics
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, spring_force_vec, Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

def referencePos_q3(t):
    """
    Generate a reference position for springPos 

    Args:
        t (float): current time (s)
    Returns:
        np.array: 3-vector of reference position
    """
    return np.array([0, 1, 1])

def calculate_spring_wrench(thetalist, Slist, stiffness, restLength, springPos):
    """
    Calculate the 6-vector spring wrench acting on the end-effector.

    Args:
        thetalist (np.array): n-vector of joint angles (rad)
        Mlist (np.array): 8 frames of link configuration at home pose
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
        springPOs (np.array): 3-vector of spring position in {s} frame
    Returns:
        np.array: 6-vector of spring forces and torque acting on the robot. Expressed in end-effector frame.
    """
    # Get end effector transformation matrix for current configuration

    eePos = mr.FKinSpace(ur5.M_EE, Slist, thetalist)
    # print(f"eePos = {eePos}")
    # Extract position vector (first 3 elements of last column)
    p = np.array(eePos[:3,3])

    # Calculate spring length
    spring_length = np.linalg.norm(p - springPos) - restLength
    # print(f"spring_length = {spring_length}")
    # print(f"expected spring force = {stiffness * spring_length}")

    # Calculate spring force vector in {s} frame
    spring_force = stiffness * spring_length * (springPos - p) / np.linalg.norm(p - springPos)
    # print(f"spring_force = {spring_force}")
    # print(f"norm = {np.linalg.norm(spring_force)}")

    # Convert to end effector frame: T_{ee}^{s} * F_{s}
    spring_force_ee = mr.TransInv(eePos) @ np.array([*spring_force, 1]).T
    # print(f"spring_force_ee = {spring_force_ee}")
    # print(f"norm = {np.linalg.norm(spring_force_ee[:3])}")

    spring_wrench_ee = np.array([0, 0, 0, *spring_force_ee[:3]])
    return spring_wrench_ee

In [73]:
q3_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q3_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g_q3 = np.array([0, 0, 0])

q3_thetamat, _ = puppet_q3(q3_thetalist0, q3_dthetalist0, g_q3, ur5.Mlist, ur5.Slist, ur5.Glist, 10, 0.01, 0, 10, 0)
# Save to csv file
print(q3_thetamat)
np.savetxt('q3_thetamat.csv', q3_thetamat, delimiter=',')

  1%|▏         | 14/1000 [00:00<00:07, 139.50it/s]

Iteration 0
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 1
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 2
eePos = [[

  4%|▍         | 42/1000 [00:00<00:07, 130.75it/s]

Iteration 28
eePos = [[-9.94003536e-01 -5.70898844e-04  1.09346440e-01  8.33292455e-01]
 [ 1.09347916e-01 -5.70133840e-03  9.93987187e-01  1.33992793e-01]
 [ 5.59549223e-05  9.99983584e-01  5.72957713e-03 -9.94418083e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6288391842414407
expected spring force = 16.288391842414406
spring_force = [-8.33292455  8.66007207 10.99441808]
norm = 16.288391842414402
spring_force_ee = [10.04418188 11.0503007   7.53608348  1.        ]
norm = 16.7268433760141
[ 0.          0.          0.         10.04418188 11.0503007   7.53608348]
Iteration 29
eePos = [[-9.93095858e-01 -7.03114464e-04  1.17303545e-01  8.33499591e-01]
 [ 1.17305632e-01 -6.53504644e-03  9.93074359e-01  1.29633958e-01]
 [ 6.83391669e-05  9.99978399e-01  6.57240684e-03 -1.05945117e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6356568147034631
expected spring force = 16.356568147034633
spring_force = [-8.

  7%|▋         | 69/1000 [00:00<00:07, 125.79it/s]

...

Iteration 941
eePos = [[ 0.27817304  0.93959501 -0.19945171  0.6333065 ]
 [ 0.06473095  0.18883759  0.97987258  0.37621523]
 [ 0.95834736 -0.28548483 -0.00829135  0.0774521 ]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2811241930577606
expected spring force = 12.811241930577605
spring_force = [-6.33306504  6.23784774  9.22547899]
norm = 12.811241930577605
spring_force_ee = [ 7.20855972 -8.05029432  7.05725813  1.        ]
norm = 12.90641174624517
[ 0.          0.          0.          7.20855972 -8.05029432  7.05725813]
Iteration 942
eePos = [[ 0.28287887  0.93775118 -0.20150003  0.63556497]
 [ 0.06560876  0.19067074  0.97945912  0.37946871]
 [ 0.95690911 -0.29028845 -0.00758797  0.08392551]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2760072053773586
expected spring force = 12.760072053773586
spring_force = [-6.35564969  6.20531289  9.16074494]
norm = 12.760072053773587
spring_force_ee = [ 7.09025072 -8.08009766  7.04603153  1.       

 99%|█████████▉| 992/1000 [00:07<00:00, 126.74it/s]

Iteration 967
eePos = [[ 0.40508658  0.89395922 -0.19168146  0.6986719 ]
 [ 0.05209038  0.18674614  0.98102623  0.42045432]
 [ 0.91279322 -0.40738532  0.02908168  0.19863845]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2108657892194368
expected spring force = 12.108657892194367
spring_force = [-6.98671899  5.79545684  8.01361552]
norm = 12.108657892194366
spring_force_ee = [ 4.30019531 -9.05037199  6.9734381   1.        ]
norm = 12.207774240362742
[ 0.          0.          0.          4.30019531 -9.05037199  6.9734381 ]
Iteration 968
eePos = [[ 0.41013076  0.89214893 -0.18937541  0.70170256]
 [ 0.05006144  0.18530779  0.98140454  0.42035097]
 [ 0.91065175 -0.4119846   0.03133817  0.20119732]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2109769632207996
expected spring force = 12.109769632207996
spring_force = [-7.01702562  5.79649027  7.98802682]
norm = 12.109769632207996
spring_force_ee = [ 4.19453928 -9.09806842  6.98192978  1.      

100%|██████████| 1000/1000 [00:08<00:00, 123.99it/s]

Iteration 993
eePos = [[ 0.53494267  0.83929048 -0.09709698  0.79636298]
 [-0.02938807  0.13333714  0.99063492  0.3665664 ]
 [ 0.84437709 -0.5270794   0.09599285  0.20944448]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2885690312566285
expected spring force = 12.885690312566284
spring_force = [-7.96362977  6.33433601  7.9055552 ]
norm = 12.885690312566284
spring_force_ee = [  1.63694444 -10.61291463   7.50122145   1.        ]
norm = 13.09892618814224
[  0.           0.           0.           1.63694444 -10.61291463
   7.50122145]
Iteration 994
eePos = [[ 0.53976252  0.83677618 -0.09198938  0.80077962]
 [-0.03355289  0.13057276  0.99087081  0.36211983]
 [ 0.8411484  -0.53174841  0.09855454  0.20741427]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2947321128503322
expected spring force = 12.947321128503322
spring_force = [-8.00779616  6.37880173  7.92585728]
norm = 12.94732112850332
spring_force_ee = [  1.53593989 -10.68946144   7.532737

In [71]:
q3_thetamat_damp, _ = puppet_q3(q3_thetalist0, q3_dthetalist0, g_q3, ur5.Mlist, ur5.Slist, ur5.Glist, 10, 0.01, 2, 10, 0)
# Save to csv file
# print(q3_thetamat_damp)
np.savetxt('q3_thetamat_damp.csv', q3_thetamat_damp, delimiter=',')

  1%|▏         | 14/1000 [00:00<00:07, 135.26it/s]

Iteration 0
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 1
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 2
eePos = [[

  4%|▍         | 42/1000 [00:00<00:07, 136.21it/s]

Iteration 28
eePos = [[-9.95672442e-01 -1.45452148e-02  9.17868499e-02  8.28434363e-01]
 [ 9.17821634e-02  9.90816304e-04  9.95778616e-01  1.36255849e-01]
 [-1.45747578e-02  9.99893722e-01  3.48462794e-04 -7.26162943e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6071288580702863
expected spring force = 16.071288580702863
spring_force = [-8.28434363  8.63744151 10.72616294]
norm = 16.071288580702863
spring_force_ee = [ 9.69620953 10.93860199  7.63262868  1.        ]
norm = 16.490194460600588
[ 0.          0.          0.          9.69620953 10.93860199  7.63262868]
Iteration 29
eePos = [[-9.95076265e-01 -1.38514929e-02  9.81395043e-02  8.28723970e-01]
 [ 9.81348545e-02  1.01662931e-03  9.95172607e-01  1.32196952e-01]
 [-1.38843978e-02  9.99903547e-01  3.47690550e-04 -7.71208381e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6124685577325635
expected spring force = 16.124685577325636
spring_force = [-

  7%|▋         | 70/1000 [00:00<00:06, 136.04it/s]

...

Iteration 939
eePos = [[ 0.99906563  0.03508875  0.02523175 -0.00775856]
 [ 0.01995832  0.14325786 -0.98948413 -0.72172744]
 [-0.03833441  0.98906318  0.14242369 -0.52842826]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.302281155531179
expected spring force = 23.02281155531179
spring_force = [ 0.07758557 17.21727444 15.28428256]
norm = 23.022811555311787
spring_force_ee = [ -0.16287417  18.21266761 -15.49609971   1.        ]
norm = 23.91352954755039
[  0.           0.           0.          -0.16287417  18.21266761
 -15.49609971]
Iteration 940
eePos = [[ 0.99909262  0.03518512  0.02399876 -0.00854475]
 [ 0.01873764  0.14288991 -0.9895612  -0.72163215]
 [-0.03824701  0.98911298  0.14210097 -0.52852877]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.3022794099546893
expected spring force = 23.022794099546893
spring_force = [ 0.0854475  17.21632149 15.28528773]
norm = 23.02279409954689
spring_force_ee = [ -0.17480933  18.20811084 -15.50128863

 98%|█████████▊| 983/1000 [00:07<00:00, 130.48it/s]

Iteration 966
eePos = [[ 0.99932046  0.03669162 -0.00351467 -0.0257944 ]
 [-0.00834937  0.13245861 -0.99115337 -0.717678  ]
 [-0.03590147  0.99050918  0.13267495 -0.53262756]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.3021794671533025
expected spring force = 23.021794671533023
spring_force = [ 0.25794397 17.17678001 15.32627558]
norm = 23.021794671533026
spring_force_ee = [ -0.43521995  18.08907502 -15.63307047   1.        ]
norm = 23.91227600770565
[  0.           0.           0.          -0.43521995  18.08907502
 -15.63307047]
Iteration 967
eePos = [[ 0.99931631  0.03671158 -0.00437766 -0.02632093]
 [-0.00919273  0.13203743 -0.9912021  -0.71747896]
 [-0.03581058  0.99056468  0.13228464 -0.53283458]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.3021747584304464
expected spring force = 23.021747584304464
spring_force = [ 0.26320928 17.17478964 15.32834575]
norm = 23.02174758430447
spring_force_ee = [ -0.44314459  18.0846032  -15.637931

100%|██████████| 1000/1000 [00:07<00:00, 131.03it/s]

Iteration 993
eePos = [[ 0.99911563  0.03636847 -0.0211021  -0.03605226]
 [-0.02537046  0.12121397 -0.99230213 -0.7115994 ]
 [-0.03353064  0.99195993  0.12202946 -0.53905885]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.3020804593190904
expected spring force = 23.020804593190903
spring_force = [ 0.36052257 17.11599403 15.39058855]
norm = 23.020804593190906
spring_force_ee = [ -0.59020145  17.97694819 -15.75484121   1.        ]
norm = 23.910960367668117
[  0.           0.           0.          -0.59020145  17.97694819
 -15.75484121]
Iteration 994
eePos = [[ 0.99910824  0.03632636 -0.02152006 -0.03627211]
 [-0.02576698  0.12081533 -0.99234053 -0.71135673]
 [-0.03344817  0.99201011  0.12164361 -0.53932078]
 [ 0.          0.          0.          1.        ]]
spring_length = 2.302078618948431
expected spring force = 23.020786189484312
spring_force = [ 0.36272111 17.11356727 15.39320777]
norm = 23.02078618948431
spring_force_ee = [ -0.59357107  17.97324731 -15.758890

From the simulation in part3a, since the total energy of the system is mostly conserved, we expect the robot to swing (under the effect of the spring) to about the same level. The motion in the simulation makes sense in this way. The total energy, when simulated at a reasonable timestep, should be roughly conserved over time. Now, if the spring constant becomes too large, the simulation becomes again very unstable and the robot spins out of control.

For the first video, part3a, there's no damping, and stiffness = 10 N/m. For the second video with damping, damping = 2 Nm/rad, and stiffness = 10 N/m

## Part 4: A moving spring. 

Use the joint damping and spring stiffness from Part 3(b), a spring
restLength of zero, and zero gravity. Now set referencePos to return a sinusoidal motion of
springPos. springPos should sinusoidally oscillate along a line, starting from one endpoint at (1, 1, 1)
to another endpoint at (1, −1, 1), completing two full back-and-forth cycles in 10 s. Simulate with
the robot starting at the home configuration for t = 10 s with dt = 0.01 s and create a movie of
the simulation

In [78]:
def referencePos_q4(t):
    """
    Generate a reference position for springPos that oscillates sinusoidally along a line
    Args:
        t (float): current time (s)
    Returns:
        np.array: 3-vector of reference position
    """
    # Start point: (1, 1, 1)
    # End point: (1, -1, 1)
    # 2 full cycles in 10s means angular frequency = 4π/10 rad/s
    
    # Only y-coordinate varies, x and z stay constant at 1
    omega = 4 * np.pi / 10  # angular frequency for 2 cycles in 10s
    y = np.cos(omega * t)  # oscillates between 1 and -1
    ref_point = np.array([1, y, 1])
    print(f"SpringPos = {ref_point}")
    return ref_point

def puppet_q4(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q3: Adding a static spring.

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        # Calculate damping
        print(f"Iteration {i}")
        tau_damping = - damping * dthetalist
        # Calculate spring force
        spring_force_vec = calculate_spring_wrench(thetalist, Slist, stiffness, restLength, referencePos_q4(i*dt))
        print(spring_force_vec)
        # Forward dynamics
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, spring_force_vec, Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [80]:
q4_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q4_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g_q4 = np.array([0, 0, 0])

q4_thetamat, _ = puppet_q4(q4_thetalist0, q4_dthetalist0, g_q4, ur5.Mlist, ur5.Slist, ur5.Glist, 10, 0.01, 2, 10, 0)
# Save to csv file
print(q4_thetamat)
np.savetxt('q4_thetamat.csv', q4_thetamat, delimiter=',')

  1%|▏         | 14/1000 [00:00<00:07, 132.52it/s]

Iteration 0
SpringPos = [1. 1. 1.]
[ 0.        0.        0.       -1.01025  10.060401  7.89405 ]
Iteration 1
SpringPos = [1.         0.99992104 1.        ]
[ 0.          0.          0.         -1.01025    10.060401    7.89326044]
Iteration 2
SpringPos = [1.         0.99968419 1.        ]
[ 0.          0.          0.         -1.01040113 10.06326505  7.89224149]
Iteration 3
SpringPos = [1.         0.99928947 1.        ]
[ 0.          0.          0.         -1.01012001 10.06883668  7.89099452]
Iteration 4
SpringPos = [1.         0.99873696 1.        ]
[ 0.          0.          0.         -1.00879643 10.07700704  7.88952206]
Iteration 5
SpringPos = [1.         0.99802673 1.        ]
[ 0.          0.          0.         -1.005988   10.08768412  7.88782813]
Iteration 6
SpringPos = [1.        0.9971589 1.       ]
[ 0.          0.          0.         -1.00133246 10.10079287  7.88591822]
Iteration 7
SpringPos = [1.         0.99613361 1.        ]
[ 0.          0.          0.         -0.994543   

  4%|▍         | 41/1000 [00:00<00:07, 127.43it/s]

Iteration 26
SpringPos = [1.        0.9470983 1.       ]
[ 0.          0.          0.         -0.30094926 10.79148998  7.81912564]
Iteration 27
SpringPos = [1.         0.94299054 1.        ]
[ 0.          0.          0.         -0.23128371 10.84307719  7.81534793]
Iteration 28
SpringPos = [1.         0.93873386 1.        ]
[ 0.          0.          0.         -0.15817368 10.89579022  7.8116207 ]
Iteration 29
SpringPos = [1.         0.93432894 1.        ]
[ 0.          0.          0.         -0.08161707 10.94954928  7.80795482]
Iteration 30
SpringPos = [1.         0.92977649 1.        ]
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.61396891e-03
  1.10042728e+01  7.80436078e+00]
Iteration 31
SpringPos = [1.         0.92507721 1.        ]
[ 0.          0.          0.          0.08183336 11.05987757  7.8008486 ]
Iteration 32
SpringPos = [1.         0.92023185 1.        ]
[ 0.          0.          0.          0.16872041 11.11627885  7.79742785]
Iteration 33
SpringPos = [1.         0.9

  7%|▋         | 68/1000 [00:00<00:07, 128.15it/s]

...

Iteration 931
SpringPos = [1.         0.64705596 1.        ]
[  0.           0.           0.          -3.73206185  12.98693767
 -20.85170167]
Iteration 932
SpringPos = [1.         0.65658576 1.        ]
[  0.           0.           0.          -3.57186404  13.04454838
 -20.92011454]
Iteration 933
SpringPos = [1.         0.66601187 1.        ]
[  0.           0.           0.          -3.40815775  13.10859182
 -20.98349007]
Iteration 934
SpringPos = [1.         0.67533281 1.        ]
[  0.           0.           0.          -3.24109353  13.17913537
 -21.04161373]
Iteration 935
SpringPos = [1.         0.68454711 1.        ]
[  0.           0.           0.          -3.07083529  13.25622407
 -21.09427598]
Iteration 936
SpringPos = [1.         0.69365331 1.        ]
[  0.           0.           0.          -2.89755961  13.33987992
 -21.14127323]
Iteration 937
SpringPos = [1.         0.70264997 1.        ]
[  0.           0.           0.          -2.7214549   13.43010131
 -21.1824089 ]
Iterat

 98%|█████████▊| 981/1000 [00:07<00:00, 128.86it/s]

[  0.           0.           0.           1.04286522  16.40815146
 -20.57351411]
Iteration 958
SpringPos = [1.         0.86392342 1.        ]
[  0.           0.           0.           1.2248654   16.59700988
 -20.46813932]
Iteration 959
SpringPos = [1.         0.87018375 1.        ]
[  0.           0.           0.           1.40467299  16.78733829
 -20.35603323]
Iteration 960
SpringPos = [1.         0.87630668 1.        ]
[  0.           0.           0.           1.5821144   16.97878516
 -20.23735107]
Iteration 961
SpringPos = [1.         0.88229123 1.        ]
[  0.           0.           0.           1.75702698  17.1710008
 -20.11226377]
Iteration 962
SpringPos = [1.         0.88813645 1.        ]
[  0.           0.           0.           1.92925939  17.3636388
 -19.98095718]
Iteration 963
SpringPos = [1.         0.89384142 1.        ]
[  0.           0.           0.           2.0986719   17.55635741
 -19.84363131]
Iteration 964
SpringPos = [1.         0.89940525 1.        ]
[  0.   

100%|██████████| 1000/1000 [00:07<00:00, 126.72it/s]

[  0.           0.           0.           4.78251063  20.98512184
 -16.2254154 ]
Iteration 984
SpringPos = [1.         0.97985505 1.        ]
[  0.           0.           0.           4.88053167  21.11865047
 -16.02260328]
Iteration 985
SpringPos = [1.         0.98228725 1.        ]
[  0.           0.           0.           4.97537985  21.24756026
 -15.81977384]
Iteration 986
SpringPos = [1.         0.98456433 1.        ]
[  0.           0.           0.           5.06712765  21.37179068
 -15.61718698]
Iteration 987
SpringPos = [1.         0.98668594 1.        ]
[  0.           0.           0.           5.1558511   21.49129321
 -15.41509572]
Iteration 988
SpringPos = [1.         0.98865174 1.        ]
[  0.           0.           0.           5.24162919  21.6060308
 -15.21374574]
Iteration 989
SpringPos = [1.         0.99046143 1.        ]
[  0.           0.           0.           5.32454341  21.71597748
 -15.01337509]
Iteration 990
SpringPos = [1.        0.9921147 1.       ]
[  0.     

Run the codes above to generate csv trajectory for part 4.